In [1]:
import pandas as pd
import plotly.io as pio
from pathlib import Path

# # Get the string name of the currently active default renderer
# default_renderer_name = pio.renderers.default

# # Use that name to access the renderer object and set its config
# # This is the line that fixes the AttributeError
# pio.renderers[default_renderer_name].config = {'doubleClickDelay': 300}

# print(f"Plotly environment configured for the '{default_renderer_name}' renderer.")
# print(f"Double-click delay is now set to {pio.renderers[default_renderer_name].config['doubleClickDelay']}ms.")

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 3000)

download_path = Path.home() / "Downloads"  
OHLCV_file_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_clean_stocks_etfs.parquet'

df_OHLCV = pd.read_parquet(OHLCV_file_path, engine='pyarrow')

In [ ]:
df_OHLCV.info()

In [ ]:
# import pandas as pd
# import plotly.graph_objects as go
# from datetime import datetime
# import numpy as np
# import ipywidgets as widgets
# from IPython.display import display, Markdown
# import pprint 

# # --- Main Plotting Function (MODIFIED) ---
# def plot_interactive_performers_widget(df_ohlcv, 
#                                        default_metric='Price',
#                                        default_period='1Y', 
#                                        default_rank_start=1,
#                                        default_rank_end=10):
#     """
#     Creates a robust interactive plot using ipywidgets for state management.
#     Metrics include Price, Sharpe, and Return/ATR.
#     Allows for selecting tickers by a slice of their rank (e.g., 10th to 20th).
    
#     Returns a list-like container. The first element (index 0) of this container 
#     will be updated with a pandas DataFrame containing the ranks and metric values 
#     of the currently displayed tickers.
#     """
#     # 1. Data Preparation: Reshape & Calculate Intermediate Values
#     print("Reshaping data and calculating intermediate metrics (ATR)...")
#     if not isinstance(df_ohlcv.index, pd.MultiIndex):
#         raise ValueError("The input DataFrame does not have the expected 'long' format with a (Ticker, Date) MultiIndex.")
    
#     df_close = df_ohlcv['Adj Close'].unstack(level=0)
#     df_high = df_ohlcv['Adj High'].unstack(level=0)
#     df_low = df_ohlcv['Adj Low'].unstack(level=0)
#     df_close.index = pd.to_datetime(df_close.index)
#     df_high.index = pd.to_datetime(df_high.index)
#     df_low.index = pd.to_datetime(df_low.index)

#     high_low = df_high - df_low
#     high_prev_close = abs(df_high - df_close.shift(1))
#     low_prev_close = abs(df_low - df_close.shift(1))
#     tr = np.maximum(high_low, np.maximum(high_prev_close, low_prev_close))
#     df_atr = tr.ewm(alpha=1/14, adjust=False).mean()
#     print("Data successfully reshaped and ATR calculated.")
    
#     # 2. Pre-computation Phase
#     print("Starting pre-computation for all metrics across all periods...")
#     end_date = df_close.index.max()
#     metrics = ['Price', 'Sharpe', 'Return/ATR']
#     periods = {
#         '1D': end_date - pd.DateOffset(days=1), '5D': end_date - pd.DateOffset(days=5),
#         '3M': end_date - pd.DateOffset(months=3), '6M': end_date - pd.DateOffset(months=6),
#         'YTD': datetime(end_date.year, 1, 1), '1Y': end_date - pd.DateOffset(years=1),
#         '5Y': end_date - pd.DateOffset(years=5),
#     }
#     all_rankings = {metric: {} for metric in metrics}
#     all_metric_values = {metric: {} for metric in metrics} # <<< NEW: To store metric values
#     period_normalized_data = {}
    
#     for label, start_date in periods.items():
#         period_close = df_close.loc[start_date:end_date]
#         period_atr = df_atr.loc[start_date:end_date]
#         if period_close.empty or len(period_close) < 2: continue
        
#         # Price
#         first_prices, last_prices = period_close.bfill().iloc[0], period_close.ffill().iloc[-1]
#         price_perf = (last_prices / first_prices).dropna().sort_values(ascending=False)
#         all_metric_values['Price'][label] = price_perf
#         all_rankings['Price'][label] = price_perf.index.tolist()
        
#         # Sharpe
#         daily_returns = period_close.pct_change()
#         mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
#         sharpe_ratio = (mean_returns / std_returns).fillna(0)
#         annualized_sharpe = (sharpe_ratio * np.sqrt(252)).sort_values(ascending=False)
#         all_metric_values['Sharpe'][label] = annualized_sharpe
#         all_rankings['Sharpe'][label] = annualized_sharpe.index.tolist()
        
#         # Return/ATR
#         mean_atr = period_atr.mean()
#         return_on_atr = (mean_returns / mean_atr).fillna(0).sort_values(ascending=False)
#         all_metric_values['Return/ATR'][label] = return_on_atr
#         all_rankings['Return/ATR'][label] = return_on_atr.index.tolist()

#         period_normalized_data[label] = period_close.div(first_prices)
#     print("Pre-computation finished.")

#     # 3. Create ipywidgets Controls & Data Holder
#     rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
#     max_traces = 50 
#     metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
#     period_dropdown = widgets.Dropdown(options=list(periods.keys()), value=default_period, description='Period:')
#     rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
#     rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
#     update_button = widgets.Button(description="Update Chart", button_style='primary')
#     ticker_list_output = widgets.Output()
    
#     # <<< NEW: Container to hold the results DataFrame for external access
#     results_container = [None] 

#     # 4. Create a FigureWidget
#     fig = go.FigureWidget()
#     for i in range(max_traces):
#         fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))

#     # 5. Define the Update Logic (MODIFIED)
#     def update_plot(button_click):
#         metric, period, rank_start, rank_end = metric_dropdown.value, period_dropdown.value, rank_start_dropdown.value, rank_end_dropdown.value
#         ticker_list_output.clear_output()
#         if rank_start > rank_end:
#             with ticker_list_output: print("Error: 'Rank Start' must be less than or equal to 'Rank End'.")
#             results_container[0] = None # Clear results on error
#             return
        
#         ranked_tickers = all_rankings[metric].get(period, [])
#         metric_values = all_metric_values[metric].get(period)
        
#         tickers_to_display = ranked_tickers[rank_start-1:rank_end]
#         norm_data = period_normalized_data.get(period)
        
#         with fig.batch_update():
#             for i in range(max_traces):
#                 trace = fig.data[i]
#                 if i < len(tickers_to_display) and norm_data is not None:
#                     ticker = tickers_to_display[i]
#                     if ticker in norm_data.columns:
#                         trace.x, trace.y, trace.name = norm_data[ticker].index, norm_data[ticker], ticker
#                         trace.visible, trace.showlegend = True, True
#                     else:
#                         trace.visible, trace.showlegend = False, False
#                 else:
#                     trace.visible, trace.showlegend = False, False

#         # <<< NEW: Build and store the results DataFrame
#         if tickers_to_display and metric_values is not None:
#             results_df = pd.DataFrame({
#                 'Rank': range(rank_start, rank_start + len(tickers_to_display)),
#                 'MetricValue': metric_values.loc[tickers_to_display].values,
#                 'Period': period
#             }, index=pd.Index(tickers_to_display, name='Ticker'))
#             results_container[0] = results_df
#         else:
#             results_container[0] = None # No data to display, set to None

#         with ticker_list_output:
#             print("Currently Displayed Tickers (Ranked):")
#             pprint.pprint(tickers_to_display, width=120, compact=True)
#             if results_container[0] is not None:
#                 print("\nMetric Values:")
#                 with pd.option_context('display.max_rows', 20):
#                     display(results_container[0])

#     # 6. Set up Initial View and Layout
#     update_plot(None)
#     fig.update_layout(title_text='Top Performing Tickers (Normalized to 1)', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers', height=700, margin=dict(t=50))
#     fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
#     update_button.on_click(update_plot)
    
#     # 7. Display the UI
#     controls = widgets.HBox([metric_dropdown, period_dropdown, rank_start_dropdown, rank_end_dropdown, update_button])
#     ui_container = widgets.VBox([controls, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
#     display(ui_container, fig)
    
#     # <<< MODIFIED: Return the container
#     return results_container

In [ ]:
# import pandas as pd
# import plotly.graph_objects as go
# from datetime import datetime
# import numpy as np
# import ipywidgets as widgets
# from IPython.display import display
# import pprint 

# def plot_interactive_performers_widget(df_ohlcv, 
#                                        default_metric='Price',
#                                        default_period='1Y', 
#                                        default_rank_start=1,
#                                        default_rank_end=10):
#     """
#     Creates a robust interactive plot using ipywidgets for state management.
#     Metrics include Price, Sharpe, and Return/ATR.
#     (Final corrected version).
#     """
#     # --- 1. Data Preparation: Reshape & Calculate Intermediate Values ---
#     print("Reshaping data and calculating intermediate metrics (ATR)...")
    
#     if not isinstance(df_ohlcv.index, pd.MultiIndex):
#         raise ValueError("The input DataFrame does not have the expected 'long' format with a (Ticker, Date) MultiIndex.")
    
#     df_close = df_ohlcv['Adj Close'].unstack(level=0)
#     df_high = df_ohlcv['Adj High'].unstack(level=0)
#     df_low = df_ohlcv['Adj Low'].unstack(level=0)
#     df_close.index = pd.to_datetime(df_close.index)
#     df_high.index = pd.to_datetime(df_high.index)
#     df_low.index = pd.to_datetime(df_low.index)

#     high_low = df_high - df_low
#     high_prev_close = abs(df_high - df_close.shift(1))
#     low_prev_close = abs(df_low - df_close.shift(1))
#     tr = np.maximum(high_low, np.maximum(high_prev_close, low_prev_close))
#     df_atr = tr.ewm(alpha=1/14, adjust=False).mean()
    
#     print("Data successfully reshaped and ATR calculated.")
    
#     # --- 2. Pre-computation Phase ---
#     print("Starting pre-computation for all metrics across all periods...")
#     end_date = df_close.index.max()
#     metrics = ['Price', 'Sharpe', 'Return/ATR']
#     periods = {
#         '1D': end_date - pd.DateOffset(days=1), '5D': end_date - pd.DateOffset(days=5),
#         '3M': end_date - pd.DateOffset(months=3), '6M': end_date - pd.DateOffset(months=6),
#         'YTD': datetime(end_date.year, 1, 1), '1Y': end_date - pd.DateOffset(years=1),
#         '5Y': end_date - pd.DateOffset(years=5),
#     }
#     all_rankings = {metric: {} for metric in metrics}
#     period_normalized_data = {}
#     for label, start_date in periods.items():
#         period_close = df_close.loc[start_date:end_date]
#         period_atr = df_atr.loc[start_date:end_date]
#         if period_close.empty or len(period_close) < 2: continue
#         first_prices, last_prices = period_close.bfill().iloc[0], period_close.ffill().iloc[-1]
#         all_rankings['Price'][label] = (last_prices / first_prices).dropna().sort_values(ascending=False).index.tolist()
#         daily_returns = period_close.pct_change()
#         mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
#         sharpe_ratio = (mean_returns / std_returns).fillna(0)
#         all_rankings['Sharpe'][label] = (sharpe_ratio * np.sqrt(252)).sort_values(ascending=False).index.tolist()
#         mean_atr = period_atr.mean()
#         return_on_atr = (mean_returns / mean_atr).fillna(0)
#         all_rankings['Return/ATR'][label] = return_on_atr.sort_values(ascending=False).index.tolist()
#         period_normalized_data[label] = period_close.div(first_prices)
#     print("Pre-computation finished.")

#     # --- 3. Create ipywidgets Controls & Data Holder ---
#     rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
#     max_traces = 50 
#     metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
#     period_dropdown = widgets.Dropdown(options=list(periods.keys()), value=default_period, description='Period:')
#     rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
#     rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
#     update_button = widgets.Button(description="Update Chart", button_style='primary')
#     ticker_list_output = widgets.Output()
#     _currently_displayed_tickers = []

#     # --- 4. Create a FigureWidget ---
#     fig = go.FigureWidget()
#     for i in range(max_traces):
#         fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))

#     # --- 5. Define the Update Logic ---
#     def update_plot(button_click):
#         metric, period, rank_start, rank_end = metric_dropdown.value, period_dropdown.value, rank_start_dropdown.value, rank_end_dropdown.value
#         ticker_list_output.clear_output()
#         if rank_start > rank_end:
#             with ticker_list_output: print("Error: 'Rank Start' must be less than or equal to 'Rank End'.")
#             return
#         ranked_tickers = all_rankings[metric].get(period, [])
#         tickers_to_display = ranked_tickers[rank_start-1:rank_end]
#         norm_data = period_normalized_data.get(period)

#         with fig.batch_update():
#             for i in range(max_traces):
#                 trace = fig.data[i]
                
#                 # --- THIS IS THE CORRECTED LOGIC ---
#                 # First, check if there's a ticker available for this trace index
#                 if i < len(tickers_to_display) and norm_data is not None:
#                     ticker = tickers_to_display[i]
#                     # Second, check if that ticker actually has data in the norm_data frame
#                     if ticker in norm_data.columns:
#                         trace.x, trace.y, trace.name = norm_data[ticker].index, norm_data[ticker], ticker
#                         trace.visible, trace.showlegend = True, True
#                     else: # Handle cases where a ticker was ranked but has no data
#                         trace.visible, trace.showlegend = False, False
#                 else: # If no ticker for this trace index, hide it
#                     trace.visible, trace.showlegend = False, False

#         _currently_displayed_tickers.clear()
#         _currently_displayed_tickers.extend(tickers_to_display)
#         with ticker_list_output:
#             print("Currently Displayed Tickers:")
#             pprint.pprint(tickers_to_display, width=120, compact=True) 
    
#     # --- 6. Set up Initial View and Layout ---
#     update_plot(None)
#     fig.update_layout(title_text='Top Performing Tickers (Normalized to 1)', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers', height=700, margin=dict(t=50))
#     fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
#     update_button.on_click(update_plot)
    
#     # --- 7. Display the UI ---
#     controls = widgets.HBox([metric_dropdown, period_dropdown, rank_start_dropdown, rank_end_dropdown, update_button])
#     ui_container = widgets.VBox([controls, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
#     display(ui_container, fig)
#     return _currently_displayed_tickers

In [7]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Markdown
import pprint 

# --- Main Plotting Function (MODIFIED) ---
def plot_interactive_performers_widget(df_ohlcv, 
                                       default_metric='Price',
                                       default_period='1Y', 
                                       default_rank_start=1,
                                       default_rank_end=10):
    """
    Creates a robust interactive plot using ipywidgets for state management.
    Metrics include Price, Sharpe, and Return/ATR.
    Allows for selecting tickers by a slice of their rank (e.g., 10th to 20th).
    
    Returns a list-like container. The first element (index 0) of this container 
    will be updated with a pandas DataFrame containing the ranks and metric values 
    of the currently displayed tickers.
    """
    # ... (Sections 1, 2, 3, and 4 are unchanged) ...
    # 1. Data Preparation: Reshape & Calculate Intermediate Values
    print("Reshaping data and calculating intermediate metrics (ATR)...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex):
        raise ValueError("The input DataFrame does not have the expected 'long' format with a (Ticker, Date) MultiIndex.")
    
    df_close = df_ohlcv['Adj Close'].unstack(level=0)
    df_high = df_ohlcv['Adj High'].unstack(level=0)
    df_low = df_ohlcv['Adj Low'].unstack(level=0)
    df_close.index = pd.to_datetime(df_close.index)
    df_high.index = pd.to_datetime(df_high.index)
    df_low.index = pd.to_datetime(df_low.index)

    high_low = df_high - df_low
    high_prev_close = abs(df_high - df_close.shift(1))
    low_prev_close = abs(df_low - df_close.shift(1))
    tr = np.maximum(high_low, np.maximum(high_prev_close, low_prev_close))
    df_atr = tr.ewm(alpha=1/14, adjust=False).mean()
    print("Data successfully reshaped and ATR calculated.")
    
    # 2. Pre-computation Phase
    print("Starting pre-computation for all metrics across all periods...")
    end_date = df_close.index.max()
    metrics = ['Price', 'Sharpe', 'Return/ATR']
    periods = {
        '1D': end_date - pd.DateOffset(days=1), '5D': end_date - pd.DateOffset(days=5),
        '3M': end_date - pd.DateOffset(months=3), '6M': end_date - pd.DateOffset(months=6),
        'YTD': datetime(end_date.year, 1, 1), '1Y': end_date - pd.DateOffset(years=1),
        '5Y': end_date - pd.DateOffset(years=5),
    }
    all_rankings = {metric: {} for metric in metrics}
    all_metric_values = {metric: {} for metric in metrics}
    period_normalized_data = {}
    
    for label, start_date in periods.items():
        period_close = df_close.loc[start_date:end_date]
        period_atr = df_atr.loc[start_date:end_date]
        if period_close.empty or len(period_close) < 2: continue
        
        first_prices, last_prices = period_close.bfill().iloc[0], period_close.ffill().iloc[-1]
        price_perf = (last_prices / first_prices).dropna().sort_values(ascending=False)
        all_metric_values['Price'][label] = price_perf
        all_rankings['Price'][label] = price_perf.index.tolist()
        
        daily_returns = period_close.pct_change()
        mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
        sharpe_ratio = (mean_returns / std_returns).fillna(0)
        annualized_sharpe = (sharpe_ratio * np.sqrt(252)).sort_values(ascending=False)
        all_metric_values['Sharpe'][label] = annualized_sharpe
        all_rankings['Sharpe'][label] = annualized_sharpe.index.tolist()
        
        mean_atr = period_atr.mean()
        return_on_atr = (mean_returns / mean_atr).fillna(0).sort_values(ascending=False)
        all_metric_values['Return/ATR'][label] = return_on_atr
        all_rankings['Return/ATR'][label] = return_on_atr.index.tolist()

        period_normalized_data[label] = period_close.div(first_prices)
    print("Pre-computation finished.")

    # 3. Create ipywidgets Controls & Data Holder
    rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
    max_traces = 50 
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    period_dropdown = widgets.Dropdown(options=list(periods.keys()), value=default_period, description='Period:')
    rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
    rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    results_container = [None] 

    # 4. Create a FigureWidget
    fig = go.FigureWidget()
    for i in range(max_traces):
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))

    # 5. Define the Update Logic
    def update_plot(button_click):
        metric, period, rank_start, rank_end = metric_dropdown.value, period_dropdown.value, rank_start_dropdown.value, rank_end_dropdown.value
        ticker_list_output.clear_output()
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be less than or equal to 'Rank End'.")
            results_container[0] = None
            return
        
        ranked_tickers = all_rankings[metric].get(period, [])
        metric_values = all_metric_values[metric].get(period)
        
        tickers_to_display = ranked_tickers[rank_start-1:rank_end]
        norm_data = period_normalized_data.get(period)
        
        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(tickers_to_display) and norm_data is not None:
                    ticker = tickers_to_display[i]
                    if ticker in norm_data.columns:
                        trace.x, trace.y, trace.name = norm_data[ticker].index, norm_data[ticker], ticker
                        trace.visible, trace.showlegend = True, True
                    else:
                        trace.visible, trace.showlegend = False, False
                else:
                    trace.visible, trace.showlegend = False, False

        # <<< MODIFIED: Added 'Metric' column to the DataFrame >>>
        if tickers_to_display and metric_values is not None:
            results_df = pd.DataFrame({
                'Rank': range(rank_start, rank_start + len(tickers_to_display)),
                'Metric': metric, # The new column
                'MetricValue': metric_values.loc[tickers_to_display].values,
                'Period': period
            }, index=pd.Index(tickers_to_display, name='Ticker'))
            results_container[0] = results_df
        else:
            results_container[0] = None

        with ticker_list_output:
            print("Currently Displayed Tickers (Ranked):")
            pprint.pprint(tickers_to_display, width=120, compact=True)
            # <<< REMOVED: The block that displayed the DataFrame in the UI is now gone >>>

    # 6. Set up Initial View and Layout
    update_plot(None)
    fig.update_layout(title_text='Top Performing Tickers (Normalized to 1)', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers', height=700, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    
    # 7. Display the UI
    controls = widgets.HBox([metric_dropdown, period_dropdown, rank_start_dropdown, rank_end_dropdown, update_button])
    ui_container = widgets.VBox([controls, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    
    return results_container

### **J. Welles Wilder Jr. definition ATR that captures overnight gaps.**

Let's be precise:

*   **We WILL calculate** the **True Range (TR)** for each day, which is the maximum of:
    1.  `(Today's High - Today's Low)`
    2.  `abs(Today's High - Yesterday's Close)`
    3.  `abs(Today's Low - Yesterday's Close)`
*   Then, we will typically calculate the **Average True Range (ATR)**, which is a **Exponential Moving Average** of this True Range value. This is the standard industry indicator for volatility that includes intraday and overnight movement.

### **Return/ATR Calculation**

You are correct that the ATR calculation should be tied to the `Period` dropdown, but here's how we'll do it in a standard, robust way:

**Step 1: Pre-computation (Done once for all stocks)**

First, for the entire historical dataset, we will calculate a standard **14-day Average True Range (ATR)**. This is the industry standard lookback period defined by Wilder. This step creates a new, continuous ATR data series for every single stock, just like we have an 'Adj Close' series.

**Step 2: On-the-Fly Calculation (When you click "Update Chart")**

This is where your interpretation comes in. When you make your selections in the dropdowns (e.g., `Metric: Return/ATR`, `Period: 3M`):

1.  We slice the **daily returns** data to get only the last **3 months**.
2.  We also slice the pre-computed **14-day ATR** data to get only the last **3 months**.
3.  We then calculate:
    *   **Numerator:** The `mean()` of the daily returns over those 3 months.
    *   **Denominator:** The `mean()` of the ATR values over those 3 months.
4.  The final score for the new metric is `Numerator / Denominator`.

**In short: The `Period` dropdown defines the window of data we analyze, not the lookback period of the ATR indicator itself.**

This is the best practice because it ensures we are always comparing apples to apples. We're using a consistent measure of volatility (14-day ATR) and seeing how it behaves over different analysis windows (1D, 3M, 1Y, etc.).

In [ ]:
# # --- NEW: Initialize the variable before the call ---
# results = [] 

# Call the final function and capture the returned list
# The 'plotted_tickers' list will be updated in place every time you click the button.
results = plot_interactive_performers_widget(
    df_OHLCV, 
    default_metric='Sharpe', 
    default_period='1Y', 
    default_rank_start=20, 
    default_rank_end=40
)

# print("\n--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---")
# print(plotted_tickers)

In [ ]:
print("\n--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---")
print(results[0])

In [ ]:
# 2. Access the results DataFrame
# You can run this cell at any time to see the *current* results
# based on your selections in the widget.

current_results_df = results[0]

if current_results_df is not None:
    print("Successfully retrieved the results DataFrame:")
    display(current_results_df)
    
    # You can now work with this DataFrame
    print("\nThe best performing ticker in this set is:", current_results_df.index[0])
    print("Its Sharpe Ratio was:", current_results_df['MetricValue'].iloc[0])
else:
    print("No results to display. This might be due to an error or lack of data for the selected period.")

### Plot Tickers

In [ ]:
results_list = list(results[0].index)
print(results_list)

In [ ]:
my_tickers = ['SPY', 'VGT', 'QQQ', 'NVDA', 'GOOG', 'META', 'ORCL', 'APP', 'U', 'B']
print(my_tickers + results_list) 

In [ ]:
# --- 2. Define the list of tickers you want to keep ---
tickers_to_plot = list(set(my_tickers + results_list))



# --- 3. The Efficient Filtering Code ---
# This single line performs a fast, index-based selection.
# The result preserves the original MultiIndex structure and format.
filtered_df = df_OHLCV.loc[tickers_to_plot]


# --- 4. Verification ---
print("Filtered DataFrame created. Shape:", filtered_df.shape)
print("Filtered DataFrame Index Levels:", filtered_df.index.names)

# Verify that only the desired tickers are present
remaining_tickers = filtered_df.index.get_level_values('Ticker').unique().tolist()
print("Tickers remaining in filtered_df:", remaining_tickers)
assert set(tickers_to_plot) == set(remaining_tickers)

print("\nAll checks passed. The format is preserved correctly.")
display(filtered_df.head())

In [ ]:
# # --- NEW: Initialize the variable before the call ---
# plotted_tickers = [] 

# Call the final function and capture the returned list
# The 'plotted_tickers' list will be updated in place every time you click the button.
plotted_tickers = plot_interactive_performers_widget(
    filtered_df, 
    default_metric='Sharpe', 
    default_period='1Y', 
    default_rank_start=20, 
    default_rank_end=40
)

# print("\n--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---")
# print(plotted_tickers)

In [ ]:
print(plotted_tickers)

### Load df_finviz

In [ ]:
finviz_file_path = r'C:\Users\ping\Files_win10\python\py311\stocks\data\2025-09-23_df_finviz_merged_stocks_etfs.parquet'

In [ ]:
finviz_file_path = r'C:\Users\ping\Files_win10\python\py311\stocks\data\2025-09-23_df_finviz_merged_stocks_etfs.parquet'
df_finviz = pd.read_parquet(finviz_file_path, engine='pyarrow')

print(f'df_finviz :\n{df_finviz }')

In [ ]:
print(df_finviz.loc[plotted_tickers])

### Verification Workflow Example

In [5]:
def inspect_ticker_data(df_ohlcv, ticker, period):
    """
    Provides a detailed, transparent breakdown of performance metric calculations for a single ticker over a specific period.
    
    Returns a pandas DataFrame containing the original OHLCV data plus all intermediate columns
    used to calculate Price, Sharpe, and Return/ATR metrics. This serves as a "calculation worksheet"
    for easy verification.
    """
    # 1. Initial Data Slicing and Validation
    if ticker not in df_ohlcv.index.get_level_values(0):
        print(f"Error: Ticker '{ticker}' not found in the DataFrame.")
        return
        
    ticker_df = df_ohlcv.loc[ticker].copy()
    ticker_df.index = pd.to_datetime(ticker_df.index)
    end_date = ticker_df.index.max()
    
    periods = {
        '1D': end_date - pd.DateOffset(days=1), '5D': end_date - pd.DateOffset(days=5),
        '3M': end_date - pd.DateOffset(months=3), '6M': end_date - pd.DateOffset(months=6),
        'YTD': datetime(end_date.year, 1, 1), '1Y': end_date - pd.DateOffset(years=1),
        '5Y': end_date - pd.DateOffset(years=5),
    }
    
    if period not in periods:
        print(f"Error: Period '{period}' is not a valid option.")
        return
        
    start_date = periods[period]
    period_df = ticker_df.loc[start_date:end_date].copy()
    
    if len(period_df) < 2:
        print(f"Not enough data for ticker '{ticker}' in period '{period}'.")
        return

    # 2. Add Calculation Columns to the DataFrame
    # --- Price Performance Columns ---
    start_price = period_df['Adj Close'].bfill().iloc[0]
    period_df['Normalized Price'] = period_df['Adj Close'] / start_price

    # --- Sharpe Ratio Columns ---
    period_df['Daily Return'] = period_df['Adj Close'].pct_change()

    # --- Return/ATR Columns (matches the main function's logic) ---
    period_df['Prev Close'] = period_df['Adj Close'].shift(1)
    period_df['High-Low'] = period_df['Adj High'] - period_df['Adj Low']
    period_df['High-PrevClose'] = abs(period_df['Adj High'] - period_df['Prev Close'])
    period_df['Low-PrevClose'] = abs(period_df['Adj Low'] - period_df['Prev Close'])
    period_df['True Range'] = period_df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
    # Use the same ewm parameters as the main plotting function for consistency
    period_df['ATR'] = period_df['True Range'].ewm(alpha=1/14, adjust=False).mean()

    # 3. Calculate Final Metrics for Summary Printout
    # Price
    end_price = period_df['Adj Close'].ffill().iloc[-1]
    normalized_price = end_price / start_price
    
    # Sharpe
    mean_daily_return = period_df['Daily Return'].mean()
    std_daily_return = period_df['Daily Return'].std()
    daily_sharpe = 0 if std_daily_return == 0 else mean_daily_return / std_daily_return
    annualized_sharpe = daily_sharpe * np.sqrt(252)

    # Return/ATR
    mean_atr = period_df['ATR'].mean()
    return_on_atr = 0 if mean_atr == 0 else mean_daily_return / mean_atr

    # 4. Display Summary and DataFrame
    display(Markdown(f"### Sanity Check for Ticker: `{ticker}` | Period: `{period}`"))
    
    display(Markdown("**1. Price Performance Calculation:**"))
    print(f"   - Start Date ({period_df.index.min().date()}): {start_price:,.2f}")
    print(f"   - End Date   ({period_df.index.max().date()}): {end_price:,.2f}")
    print(f"   - Final Normalized Price: ({end_price:,.2f} / {start_price:,.2f}) = {normalized_price:.4f}\n")

    display(Markdown("**2. Sharpe Ratio Calculation:**"))
    print(f"   - Number of Trading Days: {len(period_df)}")
    print(f"   - Mean of Daily Returns: {mean_daily_return:.6f}")
    print(f"   - Std Dev of Daily Returns: {std_daily_return:.6f}")
    print(f"   - Annualized Sharpe Ratio: {annualized_sharpe:.4f}\n")
    
    display(Markdown("**3. Return/ATR Calculation:**"))
    print(f"   - Mean of Daily Returns: {mean_daily_return:.6f}")
    print(f"   - Mean ATR (14-day EWM): {mean_atr:.6f}")
    print(f"   - Return / Mean ATR: {return_on_atr:.6f}\n")

    display(Markdown("**4. Underlying Data and Calculation Worksheet:**"))
    # Define a clean column order for the final display
    display_cols = [
        'Adj Open', 'Adj High', 'Adj Low', 'Adj Close', # Original Data
        'Normalized Price', 'Daily Return',             # Metric Columns
        'Prev Close', 'True Range', 'ATR'               # ATR Intermediates
    ]
    # Filter for columns that actually exist to avoid errors if some are missing
    display_cols = [col for col in display_cols if col in period_df.columns]
    with pd.option_context('display.max_rows', 15):
        display(period_df[display_cols])
    
    return period_df

In [8]:
_ticker = 'B'
_period = '3M'

print(f"\n--- Verifying Rank #1: {_ticker} ---")
inspection_df_1 = inspect_ticker_data(
    df_OHLCV, 
    ticker=_ticker, 
    period=_period,
)


--- Verifying Rank #1: B ---


### Sanity Check for Ticker: `B` | Period: `3M`

**1. Price Performance Calculation:**

   - Start Date (2025-06-24): 20.68
   - End Date   (2025-09-24): 33.57
   - Final Normalized Price: (33.57 / 20.68) = 1.6233



**2. Sharpe Ratio Calculation:**

   - Number of Trading Days: 65
   - Mean of Daily Returns: 0.007833
   - Std Dev of Daily Returns: 0.022063
   - Annualized Sharpe Ratio: 5.6359



**3. Return/ATR Calculation:**

   - Mean of Daily Returns: 0.007833
   - Mean ATR (14-day EWM): 0.588721
   - Return / Mean ATR: 0.013305



**4. Underlying Data and Calculation Worksheet:**

,Adj Open,Adj High,Adj Low,Adj Close,Normalized Price,Daily Return,Prev Close,True Range,ATR
Date,,,,,,,,,
2025-06-24,20.54,20.890,20.45,20.68,1.000000,NaN,NaN,0.440,0.440000
2025-06-25,20.59,21.100,20.59,20.87,1.009188,0.009188,20.68,0.510,0.445000
2025-06-26,20.92,21.220,20.92,21.20,1.025145,0.015812,20.87,0.350,0.438214
2025-06-27,20.67,20.900,20.35,20.46,0.989362,-0.034906,21.20,0.850,0.467628
2025-06-30,20.48,20.830,20.39,20.82,1.006770,0.017595,20.46,0.440,0.465654
...,...,...,...,...,...,...,...,...,...
2025-09-18,29.24,30.110,29.08,30.07,1.454062,0.020014,29.48,1.030,0.716556
2025-09-19,30.16,33.200,30.15,32.99,1.595261,0.097107,30.07,3.130,0.888945
2025-09-22,34.06,35.700,34.04,35.36,1.709865,0.071840,32.99,2.710,1.019020


In [ ]:
# --- 1. Run the Main Plot to Get a Ranked List ---
# We'll use Sharpe over 1 Year and look at the top 10 performers.
_METRIC = 'Return/ATR'
_PERIOD = '3M'
_RANK_START = 1
_RANK_END = 10

print(f"--- Running main widget to get the Top {_RANK_END} tickers for '{_METRIC}' over '{_PERIOD}' ---")
# The UI will be displayed. Make sure to click "Update Chart" if defaults are different.
plotted_tickers = plot_interactive_performers_widget(
    df_OHLCV, 
    default_metric=_METRIC, 
    default_period=_PERIOD, 
    default_rank_start=_RANK_START, 
    default_rank_end=_RANK_END
)

# --- 2. Perform the Verification Check ---
print("\n" + "="*80)
print("--- STARTING VERIFICATION PROCESS ---")
print("="*80 + "\n")

# We need at least two tickers to compare. Let's check for the 1st and 5th.
if len(plotted_tickers) >= 5:
    ticker_to_check_1 = plotted_tickers[0]  # The #1 ranked ticker
    ticker_to_check_2 = plotted_tickers[4]  # The #5 ranked ticker

    print(f"\n--- Verifying Rank #1: {ticker_to_check_1} ---")
    inspection_df_1 = inspect_ticker_data(
        df_OHLCV, 
        ticker=ticker_to_check_1, 
        period=_PERIOD
    )

    print(f"\n--- Verifying Rank #5: {ticker_to_check_2} ---")
    inspection_df_2 = inspect_ticker_data(
        df_OHLCV, 
        ticker=ticker_to_check_2, 
        period=_PERIOD
    )
    
    # You can now manually compare the printed summaries above.
    # For Sharpe, look at the "Annualized Sharpe Ratio".
    # For Price, look at the "Final Normalized Price".
    # For Return/ATR, look at the "Return / Mean ATR".
    # The value for the first ticker should be greater than the value for the second.

else:
    print(f"Could not perform verification check: The plotter returned fewer than 5 tickers ({len(plotted_tickers)}).")

print("\n" + "="*80)
print("--- VERIFICATION PROCESS COMPLETE ---")
print("="*80)

In [ ]:
import os
# safe, platform-independent way
inspection_df_1.to_csv(os.path.join(download_path, f'{ticker_to_check_1}.csv'))
inspection_df_2.to_csv(os.path.join(download_path, f'{ticker_to_check_2}.csv'))

In [ ]:
cutoff = pd.Timestamp('today') - pd.DateOffset(months=3)
last_3m = df_OHLCV.loc[df_OHLCV.index >= cutoff].copy()
last_3m

In [ ]:
df_OHLCV.info()

In [ ]:
df_OHLCV.sort_index(inplace=True)
df_OHLCV.loc[(ticker_to_check_1, slice('2025-06-23', '2025-09-23')), :]

In [ ]:
# --- Example Usage after running the main plotting cell ---

# Let's say the main plot is showing tickers for 'Return/ATR' over '6M'
_period = '3M' 

# Pick the top-ranked ticker from the list returned by the main function
if plotted_tickers:
    _ticker = plotted_tickers[0] 

    # Call the new inspection function
    inspection_df = inspect_ticker_data(
        df_OHLCV, # Use the same dataframe you passed to the plot
        ticker=_ticker, 
        period=_period
    )
else:
    print("Run the main plotting widget first to populate the 'plotted_tickers' list.")

In [ ]:
_ticker = 'IREN'
_period = '3M'
_period_df = inspect_ticker_data(df_OHLCV, ticker=_ticker, period=_period)

In [ ]:
_ticker = 'IREN'
_period = '3M'
_period_df = inspect_ticker_data(df_OHLCV, ticker=_ticker, period=_period)

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

def inspect_ticker_data(df_ohlcv, ticker, period):
    """
    Performs a detailed calculation sanity check for a single ticker over a given period.

    Args:
        df_ohlcv (pd.DataFrame): The original 'long' format DataFrame with a 
                                 (Ticker, Date) MultiIndex.
        ticker (str): The ticker symbol to inspect (e.g., 'NVDA').
        period (str): The period to analyze (e.g., '1Y', '3M', '5D').
    """
    # --- 1. Data Preparation ---
    
    # Check if the ticker exists
    if ticker not in df_ohlcv.index.get_level_values(0):
        print(f"Error: Ticker '{ticker}' not found in the DataFrame.")
        return

    # Select all data for the specified ticker and create a clean DataFrame
    ticker_df = df_ohlcv.loc[ticker].copy()
    ticker_df.index = pd.to_datetime(ticker_df.index)
    
    # Define the start date based on the period
    end_date = ticker_df.index.max()
    periods = {
        '1D': end_date - pd.DateOffset(days=1), '5D': end_date - pd.DateOffset(days=5),
        '3M': end_date - pd.DateOffset(months=3), '6M': end_date - pd.DateOffset(months=6),
        'YTD': datetime(end_date.year, 1, 1), '1Y': end_date - pd.DateOffset(years=1),
        '5Y': end_date - pd.DateOffset(years=5),
    }
    
    if period not in periods:
        print(f"Error: Period '{period}' is not a valid option. Choose from {list(periods.keys())}")
        return
        
    start_date = periods[period]
    
    # Slice the DataFrame for the requested period
    period_df = ticker_df.loc[start_date:end_date].copy()

    if len(period_df) < 2:
        print(f"Not enough data for ticker '{ticker}' in period '{period}' to perform calculations.")
        return

    # --- 2. Perform and Display Calculations ---

    display(Markdown(f"### Sanity Check for Ticker: `{ticker}` | Period: `{period}`"))

    # Price Performance
    start_price = period_df['Adj Close'].bfill().iloc[0]
    end_price = period_df['Adj Close'].ffill().iloc[-1]
    normalized_price = end_price / start_price
    
    display(Markdown("**1. Price Performance Calculation:**"))
    print(f"   - Start Date ({period_df.index.min().date()}): {start_price:,.2f}")
    print(f"   - End Date   ({period_df.index.max().date()}): {end_price:,.2f}")
    print(f"   - Final Normalized Price: ({end_price:,.2f} / {start_price:,.2f}) = {normalized_price:.4f}\n")

    # Sharpe Ratio
    period_df['Daily Return'] = period_df['Adj Close'].pct_change()
    mean_daily_return = period_df['Daily Return'].mean()
    std_daily_return = period_df['Daily Return'].std()
    
    # Handle case where standard deviation is zero (flat price)
    daily_sharpe = 0 if std_daily_return == 0 else mean_daily_return / std_daily_return
    annualized_sharpe = daily_sharpe * np.sqrt(252)

    display(Markdown("**2. Sharpe Ratio Calculation:**"))
    print(f"   - Number of Trading Days: {len(period_df)}")
    print(f"   - Mean of Daily Returns: {mean_daily_return:.6f}")
    print(f"   - Std Dev of Daily Returns: {std_daily_return:.6f}")
    print(f"   - Daily Sharpe Ratio: ({mean_daily_return:.6f} / {std_daily_return:.6f}) = {daily_sharpe:.4f}")
    print(f"   - Annualized Sharpe Ratio: ({daily_sharpe:.4f} * sqrt(252)) = {annualized_sharpe:.4f}\n")
    
    # --- 3. Display Underlying Data ---
    display(Markdown("**3. Underlying Data Sample:**"))
    display(period_df[['Adj Close', 'Daily Return']].head())
    if len(period_df) > 10:
        print("   ...")
        display(period_df[['Adj Close', 'Daily Return']].tail())
        
    return period_df[['Adj Close', 'Daily Return']]



In [ ]:
_ticker = 'IREN'
_period = '3M'
_period_df = inspect_ticker_data(df_OHLCV, ticker=_ticker, period=_period)

In [ ]:
_ticker_df = df_OHLCV.loc[_ticker].copy()

##################

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import pprint # Import the pretty-print library

def plot_interactive_performers_widget(df_ohlcv, 
                                       default_metric='Price',
                                       default_period='1Y', 
                                       default_rank_start=1,
                                       default_rank_end=10):
    """
    Creates a robust interactive plot using ipywidgets for state management.
    Allows for selecting tickers by a slice of their rank (e.g., 10th to 20th).
    Returns a list that is updated with the currently displayed tickers.
    """
    # --- [Section 1 & 2: Data Prep & Pre-computation] ---
    print("Reshaping and pre-computing all performance data...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex) or 'Adj Close' not in df_ohlcv.columns:
        raise ValueError("Expected 'long' format with (Ticker, Date) MultiIndex and 'Adj Close' column.")
    df_close = df_ohlcv['Adj Close'].unstack(level=0)
    df_close.index = pd.to_datetime(df_close.index)
    end_date = df_close.index.max()
    metrics = ['Price', 'Sharpe']
    periods = {
        '1D': end_date - pd.DateOffset(days=1), '5D': end_date - pd.DateOffset(days=5),
        '3M': end_date - pd.DateOffset(months=3), '6M': end_date - pd.DateOffset(months=6),
        'YTD': datetime(end_date.year, 1, 1), '1Y': end_date - pd.DateOffset(years=1),
        '5Y': end_date - pd.DateOffset(years=5),
    }
    all_rankings = {metric: {} for metric in metrics}
    period_normalized_data = {}
    for label, start_date in periods.items():
        period_df = df_close.loc[start_date:end_date]
        if period_df.empty or len(period_df) < 2: continue
        first_prices, last_prices = period_df.bfill().iloc[0], period_df.ffill().iloc[-1]
        all_rankings['Price'][label] = (last_prices / first_prices).dropna().sort_values(ascending=False).index.tolist()
        daily_returns = period_df.pct_change()
        mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
        sharpe_ratio = (mean_returns / std_returns).fillna(0)
        all_rankings['Sharpe'][label] = (sharpe_ratio * np.sqrt(252)).sort_values(ascending=False).index.tolist()
        period_normalized_data[label] = period_df.div(first_prices)
    print("Pre-computation finished.")

    # --- 3. Create ipywidgets Controls & Data Holder ---
    rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
    max_traces = 50 
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    period_dropdown = widgets.Dropdown(options=list(periods.keys()), value=default_period, description='Period:')
    rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
    rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    
    _currently_displayed_tickers = []

    # --- 4. Create a FigureWidget ---
    fig = go.FigureWidget()
    for i in range(max_traces):
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))

    # --- 5. Define the Update Logic ---
    def update_plot(button_click):
        metric, period, rank_start, rank_end = metric_dropdown.value, period_dropdown.value, rank_start_dropdown.value, rank_end_dropdown.value
        
        ticker_list_output.clear_output()
        
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be less than or equal to 'Rank End'.")
            return
        
        ranked_tickers = all_rankings[metric].get(period, [])
        tickers_to_display = ranked_tickers[rank_start-1:rank_end]
        norm_data = period_normalized_data.get(period)

        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(tickers_to_display) and norm_data is not None:
                    ticker = tickers_to_display[i]
                    trace.x, trace.y, trace.name = norm_data[ticker].index, norm_data[ticker], ticker
                    trace.visible, trace.showlegend = True, True
                else:
                    trace.visible, trace.showlegend = False, False
        
        _currently_displayed_tickers.clear()
        _currently_displayed_tickers.extend(tickers_to_display)
        
        with ticker_list_output:
            print("Currently Displayed Tickers:")
            # --- NEW: Use compact=True to force a more horizontal layout ---
            pprint.pprint(tickers_to_display, width=120, compact=True) 

    # --- 6. Set up Initial View and Layout ---
    update_plot(None)
    fig.update_layout(
        title_text='Top Performing Tickers (Normalized to 1)',
        xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)',
        hovermode='x unified', legend_title_text='Tickers',
        height=700, margin=dict(t=50)
    )
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    
    # --- 7. Display the UI ---
    controls = widgets.HBox([metric_dropdown, period_dropdown, rank_start_dropdown, rank_end_dropdown, update_button])
    ui_container = widgets.VBox([controls, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    
    display(ui_container, fig)
    
    return _currently_displayed_tickers

In [ ]:
# --- NEW: Initialize the variable before the call ---
plotted_tickers = [] 

# Call the final function and capture the returned list
# The 'plotted_tickers' list will be updated in place every time you click the button.
plotted_tickers = plot_interactive_performers_widget(
    df_ohlcv=df_OHLCV, 
    default_metric='Sharpe', 
    default_period='1Y', 
    default_rank_start=20, 
    default_rank_end=40
)

print("\n--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---")
print(plotted_tickers)

In [ ]:
print("\n--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---")
print(plotted_tickers)